# Программирование на языке Python
## Семинар 25. Функции, замыкание и декораторы
Полезная информация для понимания темы:

1. [Функции - это объекты первого класса](https://habr.com/ru/companies/otus/articles/725374/);
2. [Декораторы](https://habr.com/ru/companies/otus/articles/727590/);
3. [Некоторые встроенные декораторы](https://www.geeksforgeeks.org/top-python-built-in-decorators-that-optimize-python-code-significantly/).

#### План занятия
1. Немного теории;
2. Некоторые полезные декораторы;
3. Практикум: настроим логи с помощью декоратора.

#### Лексическое замыкание
Это способность функции "запоминать" состояние родительской функции.

Рассмотрим на примере:

In [2]:
def enclosing_function():  # функция высшего порядка
    def show_local_state():
        return 'local state'

    return show_local_state

In [3]:
fun_from_fun = enclosing_function()  # функция сама по себе может быть возвращена другой функцией
fun_from_fun()

'local state'

In [5]:
enclosing_function

<function __main__.enclosing_function()>

In [4]:
fun_from_fun

<function __main__.enclosing_function.<locals>.show_local_state()>

In [11]:
class NotCallable:
    def __init__(self):
        self.is_collable = 'not callable'

In [12]:
nc = NotCallable()
nc()

TypeError: 'NotCallable' object is not callable

In [13]:
class Callable:
    def __init__(self):
        self.is_collable = 'not callable'

    def __call__(self):
        return "what you get when call me"

In [14]:
c = Callable()
c()

'what you get when call me'

**Задание 1**

Поменяйте предложенный ниже код, перенеся условную конструкцию в функцию.

In [ ]:
const = 48

def int1_to_str(number: int) -> str:  # baseline function
    return chr(number + const)

def int_to_str(number: int) -> str:  # while
    a = abs(number)

    if number != 0:
        result = ''
        
        while a != 0:  # a > 0
            num = a % 10
            result += int1_to_str(num)  # result = result + int1_to_str(num)
            a //= 10  # a = a // 10
        
        result = result[::-1]  # result = '-' * (some_int < 0) + result[::-1]
        
        if number < 0:
            result = '-' + result

    else:
        result = int1_to_str(number)  # result = '0'
    
    return result

def int_to_str_r(number: int) -> str:  # recursion
    if number < 10:
        return int1_to_str(number)
    else:
        return int_to_str_r(number // 10) + int1_to_str(number % 10)


# select appropriate function depending on number size
number = 892834282939493932

if number // 1e6 > 0:
    result = int_to_str(number)
else:
    result = int_to_str_r(number)

print(result)

In [15]:
def select_int_to_str(number):
    const = 48

    def int1_to_str(number: int) -> str:  # baseline function
        return chr(number + const)
    
    def int_to_str(number: int) -> str:  # while
        a = abs(number)
    
        if number != 0:
            result = ''
            
            while a != 0:  # a > 0
                num = a % 10
                result += int1_to_str(num)  # result = result + int1_to_str(num)
                a //= 10  # a = a // 10
            
            result = result[::-1]  # result = '-' * (some_int < 0) + result[::-1]
            
            if number < 0:
                result = '-' + result
    
        else:
            result = int1_to_str(number)  # result = '0'
        
        return result
    
    def int_to_str_r(number: int) -> str:  # recursion
        if number < 10:
            return int1_to_str(number)
        else:
            return int_to_str_r(number // 10) + int1_to_str(number % 10)

    if number // 1e6 > 0:
        return int_to_str
    else:
        return int_to_str_r

In [17]:
number = 892834282939493932
converter = select_int_to_str(number)

In [18]:
converter(number)

'892834282939493932'

Теперь рассмотрим непосредственно замыкание:

In [19]:
def enclosing_function(mode):  # функция высшего порядка
    if mode == 1:
        local_state = 'local state 1'
    elif mode == 2:
        local_state = 'local state 2'
        
    def show_local_state():  # функция "вытаскивает в себя" локальное состояние функции enclosing_function через return
        return local_state

    return show_local_state

In [20]:
local_func = enclosing_function(1)
local_func()

'local state 1'

In [21]:
local_func

<function __main__.enclosing_function.<locals>.show_local_state()>

In [22]:
local_func = enclosing_function(2)
local_func()

'local state 2'

In [23]:
local_func

<function __main__.enclosing_function.<locals>.show_local_state()>

**Задание 2**

Реализуйте ООП-решение задачи предыдущего семинара в парадигме функционального программирования. Обязательно используйте замыкание.

In [32]:
import numpy as np
import pandas as pd


def get_transformer(ttype='std'):
    _valid_structures = [list, tuple, np.ndarray, pd.DataFrame]
    _valid_dtypes = [int, float]

    if ttype=='std':
        def get_params(data):
            means = data.mean(axis=0)
            stds = data.std(axis=0)
            
            params = {
                'mean': means,
                'std': stds
            }
            
            return params

        def apply_params(data, params):
            data_transformed = (data - params['mean']) / params['std']
            
            return data_transformed
    else:
        def get_params(data):
            mins = data_valid.min(axis=0)
            maxs = data_valid.max(axis=0)
            
            params = {
                'min': mins,
                'max': maxs
            }
            
            return params

        def apply_params(data, params):
            data_transformed = (data - params['min']) / (params['max'] - params['min'])
            
            return data_transformed
        
    def validate_data(data):
        # check data structure
        structure = type(data)
        if structure not in _valid_structures:
            raise ValueError(f'invalid structure {structure}, use only authorized structures: {_valid_structures}!')
        
        # <data> -> np.ndarray
        if structure in [list, tuple]:
            data = np.array(data)
        elif structure == pd.DataFrame:
            data = data.values
        
        # check dtype
        if data.dtype not in _valid_dtypes:
            raise ValueError(f'invalid dtype {data.dtype}, use only authorized dtypes: {_valid_dtypes}!')

        # shape
        if data.ndim != 2:
            raise ValueError(f'invalid shape {data.ndim}, must be 2!')

        return data

    def fit(data):
        # validate data
        data_valid = validate_data(data)
        
        # compute params
        params = get_params(data_valid)
            
        # save params
        return params

    def transform(data, params) -> pd.DataFrame:
        # validate data
        data_valid = validate_data(data)

        # apply params to data
        data_transformed = apply_params(data_valid, params)

        return data_transformed

    return fit, transform

In [33]:
X = np.random.uniform(-10, 10, (100, 2))
y = np.random.normal(0, 1, 100)

X_new = np.random.uniform(-10, 10, (50, 2))

fitter, transformer = get_transformer()

In [36]:
params = fitter(X)
transformer(X_new, params)

array([[ 1.29677052,  0.26400753],
       [ 0.81529167, -1.2200083 ],
       [-1.55309641, -1.76566267],
       [ 1.68927352,  0.53520403],
       [-0.82221032,  1.22516692],
       [ 1.6851225 ,  0.10764058],
       [ 1.3922126 , -0.35950585],
       [-1.29898853,  1.44971013],
       [-0.51380792,  1.41538168],
       [-0.53332941, -1.90108028],
       [ 1.2840311 , -0.58269687],
       [-1.49009304,  1.24080317],
       [-0.60180794,  1.63433368],
       [ 1.12177642, -1.15667087],
       [-0.08793865,  1.26693057],
       [ 1.76785127,  0.11608779],
       [ 0.69383633, -0.68814814],
       [-0.31826811, -1.80094681],
       [-0.54131126, -1.20492331],
       [ 1.47667583, -0.81141903],
       [ 0.54802032, -1.17083659],
       [-0.06928106,  0.76774649],
       [-0.75896266, -0.93133588],
       [-0.34967076, -0.98360111],
       [ 0.5217566 , -0.93922845],
       [-1.15333991, -0.72768163],
       [-1.39560797, -1.60351519],
       [ 1.11134738,  0.7964338 ],
       [-0.44884502,

#### Декораторы
Декоратор - это, в сущности, функция, которая принимает на вход другую функцию и возвращает ее модификацию:

In [37]:
from datetime import datetime

def what_time_is_it():
    now = datetime.now()

    return now

In [38]:
what_time_is_it

<function __main__.what_time_is_it()>

In [39]:
what_time_is_it()

datetime.datetime(2023, 11, 27, 19, 3, 40, 903777)

In [40]:
def prettifier(func):
    def wrapper():
        now = func()
        pretty_now = now.strftime('%d.%m.%Y %H:%M:%S')
        
        return pretty_now
        
    return wrapper

Без синтаксического сахара:

In [42]:
def what_time_is_it():
    now = datetime.now()

    return now

what_time_is_it = prettifier(what_time_is_it)

С синтаксисом декораторов:

In [46]:
def what_time_is_it():
    now = datetime.now()

    return now

In [47]:
what_time_is_it

<function __main__.what_time_is_it()>

In [ ]:
def oauth():
    ...

def ssh():
    ...


@ssh
def request()

In [48]:
from datetime import datetime

@prettifier
def what_time_is_it():
    now = datetime.now()

    return now

In [ ]:
def prettifier(func):
    def wrapper():
        now = func()
        pretty_now = now.strftime('%d.%m.%Y %H:%M:%S')
        
        return pretty_now
        
    return wrapper

In [49]:
what_time_is_it

<function __main__.prettifier.<locals>.wrapper()>

In [50]:
what_time_is_it()

'27.11.2023 19:09:57'

А как с аргументами?

In [51]:
from pathlib import Path
import openpyxl

def parse_info(path: Path) -> dict:
    wb = openpyxl.load_workbook(path)
    ws = wb.active

    info_dict = {
        (row[0].value, row[1].value): {'fio': row[2].value, 'email': row[3].value}
        for row
        in ws.iter_rows()
        if row[0].value != 'Блок'
    }

    wb.close()

    return info_dict

In [52]:
info_path = Path('../Занятие 5/sources/Справочник.xlsx')

parse_info(info_path)

{('Блок 1', 'Департамент A'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 1', 'Департамент B'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 1', 'Департамент C'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент A'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент B'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент C'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент D'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент E'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент F'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 

In [74]:
import pandas as pd

def pandify(func):
    def wrapper(*args, **kwargs):
        dictlike = func(*args, **kwargs)
        
        list_to_pandas = [{'block': key[0], 'department': key[1], **value} for key, value in dictlike.items()]

        df = pd.DataFrame(list_to_pandas)

        return df

    return wrapper

In [60]:
def function_unlimited_args(*args, **kwards):
    return args, kwards

In [68]:
kwargs = {'a': 5, 'b': 6}

def multiply(a, b):
    return a * b

In [70]:
multiply(**kwargs)

30

In [62]:
args, kwargs = function_unlimited_args(1, 2, 3, 4, a='a', b='b', c='c')

In [64]:
kwargs

{'a': 'a', 'b': 'b', 'c': 'c'}

In [72]:
@pandify
def parse_info(path: Path) -> dict:
    wb = openpyxl.load_workbook(path)
    ws = wb.active

    info_dict = {
        (row[0].value, row[1].value): {'fio': row[2].value, 'email': row[3].value}
        for row
        in ws.iter_rows()
        if row[0].value != 'Блок'
    }

    wb.close()

    return info_dict

In [73]:
info_path = Path('../Занятие 5/sources/Справочник.xlsx')

parse_info(info_path)

,block,department,fio,email
0,Блок 1,Департамент A,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
1,Блок 1,Департамент B,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
2,Блок 1,Департамент C,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
3,Блок 2,Департамент A,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
4,Блок 2,Департамент B,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
5,Блок 2,Департамент C,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
6,Блок 2,Департамент D,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
7,Блок 2,Департамент E,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
8,Блок 2,Департамент F,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
9,Блок 2,Департамент G,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com


**Как быть с docstring?**

In [76]:
def func(n):
    """
    This function powers input n by 2
    """
    return n ** 2

In [77]:
func.__doc__

'\n    This function powers input n by 2\n    '

In [78]:
def divider(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result ** (1 / 2)
    return wrapper

In [79]:
@divider
def func(n):
    return n ** 2

In [82]:
func.__doc__

**Решение**

In [84]:
def func(n):
    """
    This function powers input n by 2
    """
    return n ** 2

In [85]:
import functools

def divider(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result ** (1 / 2)
    return wrapper

**Перехватим docstring сами**

In [106]:
def func(n):
    """
    This function powers input n by 2
    """
    return n ** 2

In [109]:
import functools

def divider(func):
    print(func)
    # docstring = func
    # print(docstring)
    
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result ** (1 / 2)

    # wrapper.__doc__ = docstring
    
    return wrapper

In [110]:
@divider
def func(n):
    return n ** 2

<function func at 0x1682fd940>


In [97]:
func.__doc__ = None

In [98]:
?func

Signature: func(n)
Docstring: <no docstring>
File:      /var/folders/h6/lv17v1r10lz21g745pd6774h0tgc4z/T/ipykernel_6168/855151311.py
Type:      function

#### Некоторые полезные декораторы

1. @classmethod (метод для класса, подразумевает аргумент cls);
2. @staticmethod (метод для класса, не привязан классу никак кроме namespace);
3. @abstractmethod (только обозначить наличие метода);
4. @typing.final (обозначить, что от класса нельзя наследоваться);
5. @lru_cache.

In [137]:
from abc import ABC, abstractmethod

In [138]:
class Transformer(ABC):
    @abstractmethod
    def _get_params(self, data):  # must be specified in child class
        pass

    @abstractmethod
    def _apply_params(self, data):  # must be specified in child class
        pass

In [139]:
tr = Transformer()

TypeError: Can't instantiate abstract class Transformer with abstract methods _apply_params, _get_params

In [131]:
class SomeClass:
    @classmethod
    def method_of_class(cls):
        print('class method')

    def method_of_instance(self):
        print('instance method')

In [132]:
sc = SomeClass()

In [133]:
SomeClass.method_of_class()

class method


In [134]:
sc.method_of_class()

class method


In [135]:
sc.method_of_instance()

instance method


#### Практика
Напишите декоратор для сохранения логов в файл stdout.txt. Модифицируйте проект по рассылке отчетов.

In [1]:
import sys

In [9]:
@print_catcher
def printer(number):
    print(number)

@print_catcher
def printer2(number):
    print(str(number) * number)

In [10]:
for i in range(10):
    printer(i)
    printer2(i)

In [8]:
def print_catcher(func):
    def wrapper(*args, **kwargs):
        original_stdout = sys.stdout

        with open('logs.txt', 'a') as stdout_log:
            sys.stdout = stdout_log
            func(*args, **kwargs)

        sys.stdout = original_stdout

    return wrapper

In [2]:
sys.stdout

In [ ]:
# наш код здесь